In [1]:
import torch
import pandas as pd

import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')
from torch_perturb.torch_pert_topk import PerturbedTopK

In [2]:
data_dir='/cluster/home/kheuto01/code/prob_diff_topk'
deaths = pd.read_csv(os.path.join(data_dir,'deaths_band.csv'))
deaths_TS = deaths.pivot(index='time', columns='geoid', values='death').values
perturbed_noise=0.1
num_pert_samples=50
K=25
perturbed_top_K_func = PerturbedTopK(k=K, sigma=perturbed_noise, num_samples=num_pert_samples)

In [3]:
deaths = torch.tensor(deaths_TS)
param = torch.tensor([0.2])
param.requires_grad = True

In [4]:
from torch.autograd import profiler

with torch.profiler.profile(profile_memory=True,
    activities=[
        torch.profiler.ProfilerActivity.CPU,
    ]
) as p:
    top_K_ids = perturbed_top_K_func(torch.ones_like(deaths)*param)

    top_K_ids = top_K_ids.sum(-2)
    loss = deaths - top_K_ids
    loss = loss.sum()
    loss.backward()

STAGE:2024-09-11 07:02:24 297874:297874 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-09-11 07:03:07 297874:297874 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-09-11 07:03:07 297874:297874 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [5]:
print(p.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              aten::div        31.10%       13.449s        31.10%       13.449s        6.724s      31.29 Gb      31.29 Gb             2  
                                              aten::bmm        21.50%        9.298s        21.50%        9.298s        4.649s      15.64 Gb      15.64 Gb             2  
                                            aten::copy_        14.61%        6.315s        14.61%        6.315s     350.851ms           0 b           

In [5]:
ratio_rating_TS = torch.ones_like(torch.tensor(deaths_TS), dtype=torch.float32)*param
top_K_ids = perturbed_top_K_func(ratio_rating_TS)
# Sum over k dim
top_K_ids = top_K_ids.sum(dim=-2)

loss = top_K_ids.mean()


In [8]:
loss.backward()

In [18]:
deaths_TS.shape

(64, 1620)

In [7]:
perturbed_top_K_func = PerturbedTopK(k=2, sigma=perturbed_noise, num_samples=num_pert_samples)
perturbed_top_K_func(torch.tensor([[1,2,3],[6,2,4]]))

tensor([[[0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 0., 1.]]])